In [18]:
import numpy as np

In [29]:
kT = 0.2
dE_0_1 = -1
dE_0_m1 = 0
# The probability of growing to 1
prob_g_1 = np.exp(-(dE_0_1 / kT))
# The probability of growing to -1
prob_g_m1 = np.exp(-(dE_0_m1 / kT))

fit_list = [
    {"bool": dE_0_1 < 0, "prob": prob_g_1, "spin": 1}, 
    {"bool": dE_0_m1 < 0, "prob": prob_g_m1, "spin": -1}
            ]
allow_fit = sum([i["bool"] for i in fit_list])
# Both are acceptable
if allow_fit > 1:
    # Use partition function to find which transition occurs
    prob_list = [j["prob"] for j in fit_list]
elif allow_fit > 0:
    for j in fit_list:
        if j["bool"]:
            spin = j["spin"]
# else:


1
